In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches

# ---------- LOAD DATA ---------- #

movies = pd.read_csv("movies.csv")
movies["genres"] = movies["genres"].fillna("")
movie_titles = movies["title"].tolist()

# ---------- TF-IDF ---------- #

tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(movies["genres"])


# ---------- SMALL UI HELPERS ---------- #

def line():
    print("=" * 60)

def subline():
    print("-" * 60)


# ---------- MOVIE BASED RECOMMEND ---------- #

def recommend_by_movie(user_movie, top_n=5):

    matches = movies[movies["title"].str.contains(user_movie, case=False, na=False)]

    if matches.empty:
        close = get_close_matches(user_movie, movie_titles, n=5, cutoff=0.6)
        line()
        print("MOVIE NOT FOUND")
        subline()
        if close:
            print("Did you mean:")
            for c in close:
                print("•", c)
        else:
            print("No close matches available.")
        line()
        return

    idx = matches.index[0]

    input_movie = movies.loc[idx, "title"]
    input_genres = movies.loc[idx, "genres"]

    cosine_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    sim_indices = cosine_scores.argsort()[::-1][1:top_n+1]

    line()
    print("INPUT MOVIE")
    subline()
    print("Title :", input_movie)
    print("Genre :", input_genres)
    line()

    print("\nRECOMMENDED MOVIES (WITH PROOF)\n")

    for count, i in enumerate(sim_indices, start=1):
        title = movies.loc[i, "title"]
        genre = movies.loc[i, "genres"]
        similarity = round(float(cosine_scores[i])*100, 2)

        print(f"{count}. {title}")
        print("   Genre      :", genre)
        print("   Similarity :", similarity, "%")
        subline()


# ---------- GENRE BASED RECOMMEND ---------- #

def recommend_by_genre(user_genre, top_n=5):

    user_genre = user_genre.lower()

    filtered = movies[movies["genres"].str.lower().str.contains(user_genre)]

    if filtered.empty:
        line()
        print("NO MOVIES FOUND FOR THIS GENRE")
        line()
        return

    line()
    print("TOP MOVIES FOR GENRE:", user_genre.upper())
    line()

    for idx, row in filtered.head(top_n).iterrows():
        print("Movie :", row["title"])
        print("Genre :", row["genres"])
        subline()


# ---------- MAIN ---------- #

line()
print("        MOVIE RECOMMENDATION SYSTEM")
line()

print("\nChoose Option:\n")
print("1 → Recommend by Movie Name")
print("2 → Recommend by Genre\n")

choice = input("Enter option (1 or 2): ").strip()

if choice == "1":
    movie = input("\nEnter movie name: ")
    recommend_by_movie(movie, 5)

elif choice == "2":
    genre = input("\nEnter genre (example: drama, horror): ")
    recommend_by_genre(genre, 5)

else:
    print("\nInvalid option selected.")

line()

line()

        MOVIE RECOMMENDATION SYSTEM

Choose Option:

1 → Recommend by Movie Name
2 → Recommend by Genre

Enter option (1 or 2): 2

Enter genre (example: drama, horror): action
TOP MOVIES FOR GENRE: ACTION
Movie : Heat (1995)
Genre : Action|Crime|Thriller
------------------------------------------------------------
Movie : Sudden Death (1995)
Genre : Action
------------------------------------------------------------
Movie : GoldenEye (1995)
Genre : Action|Adventure|Thriller
------------------------------------------------------------
Movie : Cutthroat Island (1995)
Genre : Action|Adventure|Romance
------------------------------------------------------------
Movie : Money Train (1995)
Genre : Action|Comedy|Crime|Drama|Thriller
------------------------------------------------------------
